<a href="https://colab.research.google.com/github/Sahanaka/fyp-main-blocks/blob/main/models/deeplearning/DNNModel6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DNN Model 6 - Text to Speech Conversion**

**This notebook will experiment the TTS Generation section.**

**The package Tenorspeech Contains SOTA models readily available for inference to be used as TTS Generators. This notebook will experiment with 2 of the most promising models (Tacotron2 & FastSpeech2). Each of these models outputs mel spectograms and we will use mel-gan model to synthesize speech. Also it should be noted that these models are trained on the LJ dataset.**

## **Setup**

In [1]:
!pip install TensorFlowTTS
!pip install tensorflow-addons
!pip install git+https://github.com/repodiac/german_transliterate

     |████████████████████████████████| 128 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 25.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 235 kB 46.3 MB/s 
     |████████████████████████████████| 1.7 MB 13.9 MB/s 
     |████████████████████████████████| 458.3 MB 10 kB/s 
     |████████████████████████████████| 3.5 MB 63.8 MB/s 
     |████████████████████████████████| 1.1 MB 46.1 MB/s 
     |████████████████████████████████| 3.1 MB 47.9 MB/s 
     |████████████████████████████████| 1.3 MB 44.4 MB/s 
     |████████████████████████████████| 20.2 MB 79.2 MB/s 
     |████████████████████████████████| 180 kB 60.4 MB/s 
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=608516 sha256=b46650d2d877a6da272354fb87481243e476af8c70bd6881271a0f301d24fd53
  Stored in directory: /root/.cache/pip/wheels/e7/7c/11/c775fffa0e1e7b05

## **Imports**

## **Load Pretrained Models**

## **Inference**

## **Audio Generation**